In [2]:
import numpy as np
import hockey.hockey_env as h_env
import gymnasium as gym
from importlib import reload
import time

In [4]:
np.set_printoptions(suppress=True)

In [5]:
reload(h_env)

c:\Users\User\anaconda3\envs\RL\Lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\User\anaconda3\envs\RL\Lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


<module 'hockey.hockey_env' from 'c:\\Users\\User\\Desktop\\hockey-env\\hockey\\hockey_env.py'>

# Normal Game Play

In [6]:
env = h_env.HockeyEnv()

have a look at the initialization condition: alternating who starts and are random in puck position

In [7]:
obs, info = env.reset()
obs_agent2 = env.obs_agent_two()
_ = env.render()

one episode with random agents

In [17]:
obs, info = env.reset()
obs_agent2 = env.obs_agent_two()

for _ in range(600000):
    env.render(mode="human")
    a1 = np.random.uniform(-1,1,4)
    a2 = np.random.uniform(-1,1,4)    
    obs, r, d, t, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d or t: break

Without rendering, it runs much faster

"info" dict contains useful proxy rewards and winning information

In [21]:
info, env.get_info_agent_two()

({'winner': 0,
  'reward_closeness_to_puck': -0.1954517698878113,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0},
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': -0.0})

Winner == 0: draw

Winner == 1: you (left player)

Winner == -1: opponent wins (right player)

In [22]:
env.close()

# Train Shooting

In [23]:
env = h_env.HockeyEnv(mode=h_env.HockeyEnv.TRAIN_SHOOTING)

In [49]:
o, info = env.reset()
_ = env.render()

for _ in range(5000000):
    env.render()
    a1 = np.random.uniform(-1,1,4)
    a2 = [0,0.,0,0] 
    obs, r, d, t , info = env.step(np.hstack([a1,a2]))
    obs_agent2 = env.obs_agent_two()
    if d or t: break

In [50]:
env.close()

# Train DEFENDING

In [64]:
env = h_env.HockeyEnv(mode=h_env.HockeyEnv.TRAIN_DEFENSE)

In [77]:
o, info = env.reset()
_ = env.render()

for _ in range(60000):
    env.render()
    a1 = np.random.uniform(-1,1,4)
    a2 = [0,0.,0,0] 
    obs, r, d, t, info = env.step(np.hstack([a1,a2]))
    print(r)
    obs_agent2 = env.obs_agent_two()
    if d or t: break

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.5999123630718938
-0.5606566384034789
-0.5248069164810676
-0.4731700374235878
-0.4286304799359794
-0.38898944329787605
-0.36189113559789704
-0.3443610078061813
-0.3404570168098295
-0.34850011419168453
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
10.0


In [78]:
env.close()

# Using discrete actions

In [79]:
import random

In [80]:
env = h_env.HockeyEnv(mode=h_env.HockeyEnv.TRAIN_SHOOTING)

In [85]:
env.reset()
for _ in range(251):
    env.render()
    a1_discrete = random.randint(0,7)
    a1 = env.discrete_to_continous_action(a1_discrete)
    a2 = [0,0.,0,0 ] 
    obs, r, d, t, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d or t: break

In [86]:
env.close()

# Hand-crafted Opponent

In [87]:
env = h_env.HockeyEnv()

In [115]:
o, info = env.reset()
_ = env.render()
player1 = h_env.BasicOpponent(weak=False)
player2 = h_env.BasicOpponent()

In [116]:
obs_buffer = []
reward_buffer=[]
obs, info = env.reset()
obs_agent2 = env.obs_agent_two()
for _ in range(251):
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, t, info = env.step(np.hstack([a1,a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)
    obs_agent2 = env.obs_agent_two()
    if d or t: break
obs_buffer = np.asarray(obs_buffer)
reward_buffer = np.asarray(reward_buffer)

In [117]:
np.mean(obs_buffer,axis=0)

array([-2.92675561, -0.01937428, -0.02277263, -0.07144933, -0.32012175,
        0.08048522,  3.13556005, -0.28572765, -0.01023405,  0.0876682 ,
       -0.03050715,  0.01437357,  0.01005807, -0.08742926, -0.79976346,
       -0.06870648,  1.25498008,  0.41832669])

In [118]:
np.std(obs_buffer,axis=0)

array([ 0.98809186,  0.82959376,  0.66705229,  3.48828048,  4.74411476,
        3.40239188,  0.50353951,  0.19661357,  0.13511938,  2.50452906,
        1.15954406,  0.66488267,  2.11218625,  1.73327489, 14.97596555,
       12.62373425,  3.54800953,  2.13216283])

If you want to use a fixed observation scaling, this might be a reasonable choice

In [119]:
scaling = [ 1.0,  1.0 , 0.5, 4.0, 4.0, 4.0,  
            1.0,  1.0,  0.5, 4.0, 4.0, 4.0,  
            2.0, 2.0, 10.0, 10.0, 4,0 ,4,0]

In [ ]:
import pylab as plt

In [ ]:
plt.plot(obs_buffer[:,2])
plt.plot(obs_buffer[:,8])

In [ ]:
plt.plot(obs_buffer[:,12])

In [ ]:
plt.plot(reward_buffer[:])

In [ ]:
np.sum(reward_buffer)

In [ ]:
info2 = env.get_info_agent_two()
info, info2, env.get_reward(info), env.get_reward_agent_two(info2)

In [120]:
env.close()

# Human Opponent

In [131]:
env = h_env.HockeyEnv(keep_mode=False)

In [138]:
player1 = h_env.HumanOpponent(env=env, player=1)
player2 = h_env.BasicOpponent(keep_mode=False)


Human Controls:
 left:			left arrow key left
 right:			arrow key right
 up:			arrow key up
 down:			arrow key down
 tilt clockwise:	w
 tilt anti-clockwise:	s
 shoot :	space


In [135]:
player1 = h_env.BasicOpponent(keep_mode=False)
player2 = h_env.HumanOpponent(env=env, player=2)


Human Controls:
 left:			left arrow key left
 right:			arrow key right
 up:			arrow key up
 down:			arrow key down
 tilt clockwise:	w
 tilt anti-clockwise:	s
 shoot :	space


In [140]:
obs, info = env.reset()

env.render()
time.sleep(1)
obs_agent2 = env.obs_agent_two()
for _ in range(900000):
    time.sleep(0.01)
    env.render()
    a1 = player1.act(obs) 
    a2 = player2.act(obs_agent2)
    obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

In [141]:
env.close()

# Check side consistency

In [142]:
env = h_env.HockeyEnv()

In [143]:
o, info = env.reset()
_ = env.render()
player1 = h_env.BasicOpponent(weak=False)
player2 = h_env.BasicOpponent(weak=False)

In [144]:
obs_buffer = []
reward_buffer=[]
obs2_buffer = []
winner_buffer = []
reward2_buffer=[]
for game in range(1000):
    obs, info = env.reset()
    obs_agent2 = env.obs_agent_two()
    for _ in range(500):
        # env.render()
        a1 = player1.act(obs)
        a2 = player2.act(obs_agent2)
#        a1 = np.random.uniform(-1,1,4)
#        a2 = np.random.uniform(-1,1,4)    
        obs, r, d, t, info = env.step(np.hstack([a1,a2]))
        info2 = env.get_info_agent_two()
        r2 = env.get_reward_agent_two(info2)
        obs_buffer.append(obs)
        obs_agent2 = env.obs_agent_two()
        obs2_buffer.append(obs_agent2)
        reward_buffer.append(r)
        reward2_buffer.append(r2)
        if d or t:
            winner_buffer.append(info["winner"])
            break
obs_buffer = np.asarray(obs_buffer)
reward_buffer = np.asarray(reward_buffer)
obs2_buffer = np.asarray(obs2_buffer)
reward2_buffer = np.asarray(reward2_buffer)

In [145]:
obs_buffer.shape

(151120, 18)

In [146]:
np.mean(obs_buffer,axis=0)

array([-2.9643861 , -0.00956122, -0.00232641, -0.05029746, -0.02222499,
        0.01300273,  2.96373563, -0.00811354, -0.00047686,  0.04610184,
        0.00434204,  0.00236478, -0.00819607, -0.01455635,  0.01599386,
       -0.01182745,  1.09710826,  1.09085495])

In [147]:
(np.std(obs_buffer,axis=0) - np.std(obs2_buffer,axis=0)) / np.std(obs_buffer,axis=0)

array([ 0.00041407, -0.00768178,  0.00121831,  0.00166323, -0.01206372,
       -0.00415247, -0.00041424,  0.00762322, -0.00121979, -0.001666  ,
        0.01191992,  0.0041353 ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.00254802, -0.00255453])

In [148]:
winner_buffer

[0,
 -1,
 0,
 0,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 0,
 1,
 0,
 -1,
 0,
 0,
 0,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 0,
 1,
 -1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 0,
 1,
 -1,
 1,
 -1,
 0,
 1,
 0,
 1,
 -1,
 -1,
 -1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 -1,
 1,
 -1,
 1,
 1,
 0,
 -1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 -1,
 1,
 1,
 0,
 0,
 1,
 -1,
 1,
 0,
 1,
 0,
 1,
 -1,
 0,
 -1,
 0,
 -1,
 0,
 0,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 0,
 -1,
 1,
 -1,
 1,
 1,
 0,
 0,
 -1,
 0,
 1,
 -1,
 1,
 0,
 -1,
 -1,
 -1,
 0,
 0,
 0,
 -1,
 -1,
 0,
 0,
 0,
 -1,
 1,
 0,
 1,
 -1,
 1,
 0,
 1,
 0,
 1,
 0,
 -1,
 -1,
 0,
 1,
 0,
 -1,
 0,
 1,
 0,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 0,
 1,
 -1,
 -1,
 0,
 -1,
 1,
 1,
 1,
 1,
 -1,
 0,
 -1,
 1,
 0,
 -1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 -1,
 -1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 -1,
 -1,
 -1,
 0,
 1,
 1,
 0,
 -1,
 -1,
 -1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 -1,
 1,
 0,
 0,
 -1,
 -1,
 -1,
 0,
 -1,
 1,
 0,
 -1,
 1,
 0,
 0,
 -1,
 0,
 0,
 -1,


In [149]:
np.mean(winner_buffer,axis=0)

0.007

In [150]:
np.std(winner_buffer,axis=0)

0.7980920999483706

In [151]:
np.sum(reward_buffer), np.sum(reward2_buffer)

(-4215.858459532259, -4484.548882941388)

In [152]:
env.close()

: 